[Open with Colab](https://colab.research.google.com/github/1never/UEC_AIX_seminar2020/blob/master/elasticsearch_dialogue.ipynb)

In [ ]:
# Elasticsearchのダウンロードと解凍
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz

# Elasticsearchの日本語形態素解析用プラグイン analysis-kuromojiのインストール
!elasticsearch-7.0.0/bin/elasticsearch-plugin install analysis-kuromoji

# Pythonのelasticsearchライブラリのインストール
!pip install elasticsearch

In [ ]:
# Elasticsearchの実行
!chown -R daemon:daemon elasticsearch-7.0.0/bin/
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# 接続テスト
!curl -X GET "localhost:9200/"

In [ ]:
# Pythonライブラリによる接続テスト
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch()
es.ping()

In [ ]:
# 対話データのダウンロード
!wget http://130.153.158.5/~inaba/unno_pair.txt

In [ ]:
# 対話データをElasticsearchにインサート
def load():
    try:
      es.delete_by_query(index='dialogue_pair', body={"query": {"match_all": {}}})
      print("既存データを削除")
    except:
      print("削除対象データなし")
      pass

    with open('./unno_pair.txt') as f:
        for i, __ in enumerate(f):
            print(i, '...', end='\r')
            __ = __.split('\t')
            query = __[0].strip()
            response = __[1].strip()
            item = {'_index':'dialogue_pair', '_type':'docs', '_source':{ 'query':query, 'response':response }}
            yield item

print(helpers.bulk(es, load()))

In [ ]:
# プログラムがエラーで落ちた場合，一時的にElasticsearchに接続できなくなりますが，一定時間経つことで接続可能になります．
es = Elasticsearch()

def get_reply(utterance, size=10):
  results = es.search(index='dialogue_pair', body={'query':{'match':{'query':utterance}}, 'size':size,})
  #out = [(result['_source']['query'], result['_source']['response'], result["_score"]) for result in results['hits']['hits']]
  return [result['_source']['response'] for result in results['hits']['hits']]

res = None
logs = []
while(True):
  u = input("\n>")
  if "exit" == u:
    break
  elif u.isdecimal() and res is not None and int(u) < len(res):
    u = res[int(u)]
  elif "back" == u:
    logs.pop()
    u = logs.pop()


  res = get_reply(u)
  logs.append(u)
  for i, l in enumerate(logs):
    print("log " + str(i) + ": " , l)
  for i, r in enumerate(res):
    print(i, r)
  